# Setup

## Load Packages

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import openai
import re
import time
from scripts.prepare_data_helpers import prepare_examples, create_query
from dotenv import load_dotenv

load_dotenv()

True

## Prepare Data

In [2]:
train1 = pd.read_csv('../../e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../../e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
dev = pd.read_csv('../../e-SNLI/dataset/esnli_dev.csv')
test = pd.read_csv('../../e-SNLI/dataset/esnli_test.csv')

train = train.dropna(subset=['Sentence1', 'Sentence2', 'Explanation_1'])
train = train.reset_index(drop=True)

np.random.seed(12345) # seed for numpy package
test_indices = list(np.random.choice(test.index.values, size=1000, replace=False))
test = test.loc[test_indices]
test = test.reset_index(drop=True)

In [3]:
np.random.seed(12345) # seed for numpy package
random.seed(12345) # seed for random package
prompts = [prepare_examples(train, size_per_class=1, include_label=True) + '\n\n' + create_query(test.loc[i], include_label=True) for i in test.index]
test['prompt'] = prompts

# Generate Predictions on Test Set

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
pred_expl = list()
pred_label = list()
for i in range(int(np.ceil(test.shape[0] / 20))):
    if i != 0:
        time.sleep(60) # API allows only 20 calls or one call with 20 prompts per minute
    indices = np.arange(i * 20, (i + 1) * 20)
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=list(test.iloc[indices].prompt),
        temperature=0.5,
        max_tokens=58,
        top_p=1,
    )
    pred_expl += [re.findall('Explanation:.+', response.choices[i].text)[0].replace('Explanation: ', '').strip() for i in range(len(response.choices))]
    pred_label += [re.findall('.+\nExplanation:', response.choices[i].text)[0].replace('\nExplanation:', '').replace('Label: ', '').strip() for i in range(len(response.choices))]

In [11]:
test_output = test
test_output['pred_explanation'] = pred_expl
test_output['pred_label'] = pred_label
test_output.to_csv('../output_data/test_predictions.csv')

In [12]:
test_output

,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,Explanation_2,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
0,4813951931.jpg#3r1e,entailment,A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.",A young man in an orange hat walks his *bicyc...,A man in an orange hat walks his *bike* down ...,9,8,A young man is still a man and a bicycle and b...,...,"9,1,2","8,1",The young man in an orange hart is the man wal...,A *young* *man* in *an* *orange* *hat* *walks...,A *man* in *an* *orange* *hat* *walks* *his* ...,"1,2,4,5,6,7,8,9,10,12","1,5,4,3,11,9,8,7,6",Statement: A man in a suit and dirty dress-sho...,The man is young and the hat is orange.,entailment
1,4678320536.jpg#1r1c,contradiction,A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,A woman standing with *smile* *on* *her* *face...,A woman stands holding her baby and *crying.*,"4,5,6,7",7,A baby is not wood.,...,13,5,the woman is either crying or has a smile,A *woman* standing with *smile* on her face an...,A *woman* stands holding her baby and *crying.*,"4,1","1,7",Statement: a white dog runs across the water.\...,"The woman is smiling and holding a wood, not a...",contradiction
2,661749711.jpg#2r1e,entailment,A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,A *dog* *jumps* to *catch* a red *ball* *outs...,the *dog* is *jumping.*,"2,7,8,1,4","3,1",Dog jumps is past tense of dog jumping.,...,"1,2","1,3","If the dog jumps to catch, the dog is jumping.",A dog *jumps* to *catch* a red ball outside.,the dog is *jumping.*,"2,4",3,Statement: A young child rides a bike with a b...,The statement is repeating the same information.,tautology
3,44904567.jpg#1r1c,contradiction,A man wielding an electric razor is gleefully ...,There is only one person present.,There cannot be one person present if one man ...,A *man* wielding an electric razor is gleefull...,There is only *one* *person* *present.*,"1,10,11","3,4,5",ANOTHER REPRESENTS ONE MORE AND CONTRAADICTORY...,...,10,"3,2",It cannot be claimed that there is only one pe...,A *man* wielding an electric razor is gleefull...,There is *only* *one* *person* present.,"1,10,11","2,3,4",Statement: A young boy wearing safety glasses ...,There are two people present in the statement.,contradiction
4,32542645.jpg#0r1n,neutral,A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,A group of people stares at a wall that is fil...,A group of people *analyze* art together,{},4,"Staring at drawings does not mean analyze art,...",...,{},"4,5",The group may have just been admiring as oppos...,A group of people stares at a wall that is fil...,A group of people *analyze* art *together*,{},"4,6",Statement: Young mother and child heading some...,Staring at a wall filled with drawings is a fo...,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2429284131.jpg#2r1c,contradiction,Three men are standing on stage performing.,Women are playing trumpets on the street.,"either there are men on stage, or women playin...",Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,Women are not men.,...,1,0,There can be either men or women.,Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,Statement: Several basketball players are grab...,"The men are standing on stage performing, whil...",contradiction
996,4460943467.jpg#0r1e,entailment,3 young man in hoods standing in the middle of...,Three hood 

In [17]:
np.mean(test_output.gold_label == test_output.pred_label)

0.675

In [19]:
print(pd.crosstab(test_output.gold_label, test_output.pred_label))

pred_label     assumption  contradiction  entailment  equivalence  equivalent  \
gold_label                                                                      
contradiction           0            299           5            0           0   
entailment              0              2         332            3           1   
neutral                 1             73         146            1           1   

pred_label     implication  inference  neutral  partial entailment  \
gold_label                                                           
contradiction            0          0       27                   0   
entailment               0          0       16                   1   
neutral                  2          4       44                   0   

pred_label     redundancy  reinforcement  repetition  speculation  synonym  \
gold_label                                                                   
contradiction           0              0           0            1        0   
entailmen